In [1]:
import pandas as pd
import numpy as np

import torch
from torch import nn
from torch import optim
from torch.utils.data import Dataset,DataLoader

In [2]:
if torch.cuda.is_available():
    device = torch.device('cuda:0')
else:
    device = torch.device('cpu')

In [3]:
full_dataset = pd.read_csv('train.csv')
# dataset = full_dataset[full_dataset['sc_w'] != 0].copy()

In [5]:
full_dataset.columns

Index(['battery_power', 'blue', 'clock_speed', 'dual_sim', 'fc', 'four_g',
       'int_memory', 'm_dep', 'mobile_wt', 'n_cores', 'pc', 'px_height',
       'px_width', 'ram', 'sc_h', 'sc_w', 'talk_time', 'three_g',
       'touch_screen', 'wifi', 'price_range'],
      dtype='object')

In [6]:
torch.manual_seed(1)
indices = torch.randperm(len(full_dataset)).tolist()
train_size = int(0.8*len(full_dataset))
dataset_train = full_dataset.iloc[indices[:train_size]]
dataset_test = full_dataset.iloc[indices[train_size:]]

In [7]:
class CreatingDatabase(Dataset):
    def __init__(self,group):
        self.data_sample = group.to_numpy()
        self.data_label = group.to_numpy()
        
    def __getitem__(self,idx):

        sample = self.data_sample[idx][0:20]
        label = self.data_label[idx][-1:]

        sample = torch.from_numpy(sample.astype(np.float32)).to(device)
        label = torch.LongTensor(label).to(device)

        # sample = torch.from_numpy(sample).float().to(device)
        # label = torch.from_numpy(label).to(device)

        return sample,label

    def __len__(self):
        return len(self.data_sample)


In [8]:
train_set = CreatingDatabase(dataset_train)
test_set = CreatingDatabase(dataset_test)

In [9]:
batch_size = 20
num_workers = 4

train_loader = DataLoader(train_set,batch_size = batch_size,shuffle = True)
test_loader = DataLoader(test_set,batch_size = batch_size,shuffle = True)

In [10]:
for i in train_loader:
    a,b = i
    print(a.dtype,b.dtype)

TypeError: can't convert np.ndarray of type numpy.object_. The only supported types are: float64, float32, float16, complex64, complex128, int64, int32, int16, int8, uint8, and bool.

# Creating NN Model

Linear ->
ReLU ->
Out ->
Softmax

In [ ]:
class PricePredictor(nn.Module):
    def __init__(self,input_size,hidden_size,out_size):
        super(PricePredictor,self).__init__()
        self.hidden = nn.Sequential(
                                    nn.Linear(input_size,hidden_size),
                                    nn.ReLU(),
                                    nn.Linear(hidden_size,hidden_size),
                                    nn.ReLU())
        self.out = nn.Linear(hidden_size,out_size)
        # self.softmax = nn.Softmax()

    def forward(self,X):
        feature = self.hidden(X)
        # output = self.softmax(self.out(feature))
        output = self.out(feature)
        
        return output

In [ ]:
input_size = 20
hidden_size = 128
out_size = 4

In [ ]:
net = PricePredictor(input_size,hidden_size,out_size).to(device)

In [ ]:
criterion = nn.CrossEntropyLoss().to(device)

In [ ]:
optimizer = optim.Adam(net.parameters(),lr=1e-3,weight_decay = 1e-4)

In [ ]:
def train(train_loader,net,epoch):

    net.train()

    epoch_loss = []
    for batch in train_loader:
        data, classification = batch
        # classification = torch.nn.functional.one_hot(classification)
        data = data.to(device)

        classification = classification.to(device)

        pred = net(data)

        print(pred.dtype, classification.dtype)
        loss = criterion(pred,classification.squeeze())

        epoch_loss.append(loss.cpu().data)

        loss.backward()
        optimizer.step()

    epoch_loss = np.asarray(epoch_loss)
    print(epoch,epoch_loss)

In [ ]:
def test(test_loader,net,epoch):

    net.eval()
    
    with torch.no_grad():
            
        epoch_loss = []
        for batch in test_loader:
            data, classification = batch

            data = data.to(device)
            classification = classification.to(device)
            
            pred = net(data)
            loss = criterion(pred,classification)

            epoch_loss.append(loss.cpu().data)

        epoch_loss = np.asarray(epoch_loss)
        print(epoch,epoch_loss)

In [ ]:
for epoch in range(10):
    train(train_loader,net,epoch)
    test(test_loader,net,epoch)
    print('-----------------------------------------------------------')

torch.float32 torch.int64
torch.float32 torch.int64
torch.float32 torch.int64
torch.float32 torch.int64
torch.float32 torch.int64
torch.float32 torch.int64
torch.float32 torch.int64
torch.float32 torch.int64
torch.float32 torch.int64
torch.float32 torch.int64
torch.float32 torch.int64
torch.float32 torch.int64
torch.float32 torch.int64
torch.float32 torch.int64
torch.float32 torch.int64
torch.float32 torch.int64
torch.float32 torch.int64
torch.float32 torch.int64
torch.float32 torch.int64
torch.float32 torch.int64
torch.float32 torch.int64
torch.float32 torch.int64
torch.float32 torch.int64
torch.float32 torch.int64
torch.float32 torch.int64
torch.float32 torch.int64
torch.float32 torch.int64
torch.float32 torch.int64
torch.float32 torch.int64
torch.float32 torch.int64
torch.float32 torch.int64
torch.float32 torch.int64
torch.float32 torch.int64
torch.float32 torch.int64
torch.float32 torch.int64
torch.float32 torch.int64
torch.float32 torch.int64
torch.float32 torch.int64
torch.float3

RuntimeError: 0D or 1D target tensor expected, multi-target not supported